In [ ]:
!pip install pandas numpy spacy scikit-learn tensorflow datasets spacy_langdetect
!pip install lckr_jupyterlab_variableinspector
!pip install spacy_fastlang

  Using cached spacy_fastlang-2.1.0-py3-none-any.whl (780 kB)
  Using cached fasttext_wheel-0.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.4 MB)
  Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)


In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import spacy
from spacy_langdetect import LanguageDetector
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder


/home/sahil/spot/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("maharshipandya/spotify-tracks-dataset")

In [3]:
#get train and test datasets
df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['train'])

In [4]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [5]:
#drop useless columns
df_train.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)
df_test.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)

In [6]:
#get prediction columns and drop from tables
target_train = df_train['popularity']
target_test = df_test['popularity']
df_train.drop(columns=['popularity'], inplace=True)
df_test.drop(columns=['popularity'], inplace=True)

In [7]:
from sklearn.compose import ColumnTransformer
#TODO FIX THIS IF BROKEN
transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['explicit', 'mode',  'track_genre', 'key', 'time_signature']),
        ('minmax_scale', MinMaxScaler(), ['duration_ms', 'loudness', 'tempo']),
    ], remainder='passthrough') 

transformer.fit(df_train)
df_train = pd.DataFrame(transformer.transform(df_train), columns=transformer.get_feature_names_out())
df_test = pd.DataFrame(transformer.transform(df_test), columns=transformer.get_feature_names_out())


  

In [ ]:
import spacy
from spacy_langdetect import LanguageDetector
import tqdm
import torch

if torch.cuda.is_available():
    spacy.require_gpu()







tqdm.pandas()
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)

def detect(lang):
    doc = nlp(lang)
    return doc._.language['language']

df_train['language'] = df_train['album_name'].progess_apply(detect)